In [1]:
import psycopg2
import keys
import cohere
import pandas as pd

pgconn = psycopg2.connect(
    host="localhost",
    dbname='caroldb',
    user='postgres',
    password=keys.pg_pwd
)
cur = pgconn.cursor()
co=cohere.Client(keys.cohere_key)

k = 5
query_text = "Tell me about the most recent hearing."

# Embed query
query_vec = co.embed(texts=[query_text], model="embed-english-v3.0", input_type = "search_query")

# Search caroldb.pages for k-most similar embeddings, return as page_results
search_query = """SELECT * FROM pages ORDER BY embedding <=> %s LIMIT %s"""
cur.execute(search_query, (query_vec, k))
page_results = pd.DataFrame(cur.fetchall(), columns=['page_id, embedding, page_number, parent_doc, text'])

# Search caroldb.transcripts for all parent_docs from page_results, return as parent_results
search_query = """SELECT * FROM transcripts WHERE hyperlink = ANY (%s)"""
hyperlinks = page_results.parent_doc.drop_duplicates().apply(lambda x: keys.hyperlink + x).to_list()
cur.execute(search_query, (hyperlinks,))
parent_results = pd.DataFrame(cur.fetchall(), columns=['transcript_id','filingname','filingdate','aenumber','attachment','hyperlink','summary'])

# Build reference_material:
#   For each doc in parent_results:
#       Add summary
#       Add all page texts from page_results
#
# Invoke model with preamble, query, and refernce material


In [4]:
import pandas as pd

def prepend(prefix, string):
    return prefix + string

s = pd.Series(['apple', 'banana', 'orange']).apply(prepend('pre'))

print(s)

TypeError: prepend() missing 1 required positional argument: 'string'